# 🎯 Hyperparameter Tuning Round 4 - Dropout Fine-Tuning
### Player Direction Prediction - Precision Dropout Optimization

**Round 3 Best:** MAE=55.34° (LR=2.07e-04, WD=1e-04, DR=0.45)

**Round 4 Strategy:**
- ✅ Fixed BS=32
- ✅ Fixed WD=1e-04  
- ✅ Fixed LR=2.07e-04 (best score: mean+std=65.87, achieved best MAE)
- 🔍 Explore DR with FINE granularity: 0.025 steps around 0.45-0.60
- **Total: 9 experiments (~1 hour on A100)**

## 1. Setup & Imports

In [1]:
# Base configuration
BASE_CONFIG = {
    'data_root': '/content/drive/MyDrive/player_direction_dataset',
    'results_dir': '/content/drive/MyDrive/hyperparameter_tuning_results_round4',
    'num_workers': 2,
    'pin_memory': True,
}

# ROUND 4: DROPOUT FINE-TUNING (9 experiments)
# LR=2.07e-04: Best score (mean+std=65.87) AND achieved best absolute MAE (55.34°)
# DR: Fine granularity around optimal region (0.45-0.60)
HYPERPARAMETER_GRID = {
    'learning_rate': [2.07e-04],  # FIXED (best from Round 3)
    'batch_size': [32],           # FIXED
    'weight_decay': [1e-04],      # FIXED

    # Fine-grained dropout exploration
    'dropout': [
        0.400,  # -0.05 from best
        0.425,
        0.450,  # Best minimum from Round 3
        0.475,
        0.500,  # Best mean from Round 3
        0.525,
        0.550,
        0.575,
        0.600
    ],  # 9 values with 0.025 step
}

FIXED_PARAMS = {
    'backbone': 'resnet34',
    'num_epochs': 100,
    'early_stopping_patience': 20,
    'early_stopping_min_delta': 0.5,
    'gradient_accumulation_steps': 2,
    'use_amp': True,
}

total_combinations = (len(HYPERPARAMETER_GRID['learning_rate']) *
                     len(HYPERPARAMETER_GRID['batch_size']) *
                     len(HYPERPARAMETER_GRID['weight_decay']) *
                     len(HYPERPARAMETER_GRID['dropout']))

assert total_combinations == 9
assert len(HYPERPARAMETER_GRID['dropout']) == 9

print(f"✅ Round 4: Dropout Fine-Tuning")
print(f"   Total: {total_combinations} experiments")
print(f"   LR (FIXED): {HYPERPARAMETER_GRID['learning_rate'][0]:.5f} ⭐")
print(f"   WD (FIXED): {HYPERPARAMETER_GRID['weight_decay'][0]:.5f}")
print(f"   BS (FIXED): {HYPERPARAMETER_GRID['batch_size'][0]}")
print(f"   DR values: {HYPERPARAMETER_GRID['dropout']}")
print(f"   Time: ~1 hour on A100")

✅ Round 4: Dropout Fine-Tuning
   Total: 9 experiments
   LR (FIXED): 0.00021 ⭐
   WD (FIXED): 0.00010
   BS (FIXED): 32
   DR values: [0.4, 0.425, 0.45, 0.475, 0.5, 0.525, 0.55, 0.575, 0.6]
   Time: ~1 hour on A100


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torchvision import transforms, models
from pathlib import Path
import json
import math
import numpy as np
from PIL import Image
from tqdm import tqdm
import time
import pandas as pd
from datetime import datetime
import itertools
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ Using device: cuda
   GPU: Tesla T4
   Memory: 15.8 GB


## 2. Configuration

## 3. Dataset Class

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class PlayerDirectionDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        if split == 'val':
            split = 'valid'

        self.root_dir = Path(root_dir) / split
        self.transform = transform

        labels_file = self.root_dir / 'labels.json'
        with open(labels_file, 'r') as f:
            labels_list = json.load(f)

        self.labels = {item['filename']: item['direction_degree'] for item in labels_list}
        self.image_files = list(self.labels.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = self.root_dir / 'images' / img_name

        image = Image.open(img_path).convert('RGB')
        angle_deg = self.labels[img_name]

        if self.transform:
            image = self.transform(image)

        angle_rad = math.radians(angle_deg)

        return {
            'image': image,
            'sin': torch.tensor(math.sin(angle_rad), dtype=torch.float32),
            'cos': torch.tensor(math.cos(angle_rad), dtype=torch.float32),
            'angle_deg': torch.tensor(angle_deg, dtype=torch.float32)
        }

print("✅ Dataset class defined")

✅ Dataset class defined


## 4. Model Architecture

In [4]:
class PlayerDirectionPredictor(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()

        resnet = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*list(resnet.children())[:-1])

        self.head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        features = self.features(x)
        sin_cos = self.head(features)
        sin_cos = nn.functional.normalize(sin_cos, p=2, dim=1)
        return sin_cos

    def predict_angle(self, x):
        sin_cos = self.forward(x)
        angle_rad = torch.atan2(sin_cos[:, 0], sin_cos[:, 1])
        angle_deg = torch.rad2deg(angle_rad) % 360
        return angle_deg

class CircularLoss(nn.Module):
    def forward(self, pred, target_sin, target_cos):
        pred_sin, pred_cos = pred[:, 0], pred[:, 1]
        loss = 1 - (pred_sin * target_sin + pred_cos * target_cos)
        return loss.mean()

print("✅ Model architecture defined")

✅ Model architecture defined


## 5. Training Functions with Mixed Precision

In [5]:
def calculate_accuracy_metrics(pred_angles, target_angles, thresholds=[15, 30, 45]):
    """Calculate accuracy at different angle thresholds"""
    errors = torch.abs(pred_angles - target_angles)
    # Handle circular nature (e.g., 359° and 1° are close)
    errors = torch.min(errors, 360 - errors)

    metrics = {}
    for threshold in thresholds:
        acc = (errors <= threshold).float().mean().item() * 100
        metrics[f'acc{threshold}'] = acc

    return metrics

def train_epoch(model, loader, criterion, optimizer, device, use_amp=True, accumulation_steps=1):
    """Train one epoch with mixed precision and gradient accumulation"""
    model.train()
    total_loss = 0
    all_pred_angles = []
    all_target_angles = []

    scaler = GradScaler(enabled=use_amp)
    optimizer.zero_grad()

    for batch_idx, batch in enumerate(loader):
        images = batch['image'].to(device)
        target_sin = batch['sin'].to(device)
        target_cos = batch['cos'].to(device)
        target_angles = batch['angle_deg'].to(device)

        # Mixed precision forward pass
        with autocast(enabled=use_amp):
            pred = model(images)
            loss = criterion(pred, target_sin, target_cos)
            # Scale loss for gradient accumulation
            loss = loss / accumulation_steps

        # Backward pass with gradient scaling
        scaler.scale(loss).backward()

        # Update weights every accumulation_steps
        if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        # Calculate metrics
        with torch.no_grad():
            pred_angles = model.predict_angle(images)
            all_pred_angles.append(pred_angles)
            all_target_angles.append(target_angles)

        total_loss += loss.item() * accumulation_steps

    all_pred_angles = torch.cat(all_pred_angles)
    all_target_angles = torch.cat(all_target_angles)

    mae = torch.mean(torch.abs(all_pred_angles - all_target_angles)).item()
    metrics = calculate_accuracy_metrics(all_pred_angles, all_target_angles)

    return total_loss / len(loader), mae, metrics

def validate(model, loader, criterion, device, use_amp=True):
    """Validate with mixed precision"""
    model.eval()
    total_loss = 0
    all_pred_angles = []
    all_target_angles = []

    with torch.no_grad():
        for batch in loader:
            images = batch['image'].to(device)
            target_sin = batch['sin'].to(device)
            target_cos = batch['cos'].to(device)
            target_angles = batch['angle_deg'].to(device)

            with autocast(enabled=use_amp):
                pred = model(images)
                loss = criterion(pred, target_sin, target_cos)

            pred_angles = model.predict_angle(images)
            all_pred_angles.append(pred_angles)
            all_target_angles.append(target_angles)

            total_loss += loss.item()

    all_pred_angles = torch.cat(all_pred_angles)
    all_target_angles = torch.cat(all_target_angles)

    mae = torch.mean(torch.abs(all_pred_angles - all_target_angles)).item()
    metrics = calculate_accuracy_metrics(all_pred_angles, all_target_angles)

    return total_loss / len(loader), mae, metrics

print("✅ Training functions defined with mixed precision")

✅ Training functions defined with mixed precision


## 6. Experiment Management

In [6]:
def get_experiment_name(params):
    """Generate unique experiment name from parameters"""
    # Use 2 decimals for dropout to avoid rounding (0.55 != 0.6)
    return f"lr{params['learning_rate']:.0e}_bs{params['batch_size']}_wd{params['weight_decay']:.0e}_dr{params['dropout']:.2f}"

def check_if_completed(results_dir, exp_name):
    """Check if experiment was already completed"""
    exp_dir = Path(results_dir) / exp_name
    if not exp_dir.exists():
        return False

    result_file = exp_dir / 'result.json'
    if result_file.exists():
        with open(result_file, 'r') as f:
            result = json.load(f)
        return result.get('completed', False)

    return False

def save_results_to_excel(results_dir, all_results):
    """Save all results to Excel file"""
    excel_file = Path(results_dir) / 'hyperparameter_tuning_results.xlsx'

    # Flatten nested params dict for DataFrame
    flattened_results = []
    for result in all_results:
        flat_result = {
            'experiment_name': result['experiment_name'],
            'learning_rate': result['params']['learning_rate'],
            'batch_size': result['params']['batch_size'],
            'weight_decay': result['params']['weight_decay'],
            'dropout': result['params']['dropout'],
            'best_val_mae': result['best_val_mae'],
            'best_val_acc15': result['best_val_acc15'],
            'best_val_acc30': result['best_val_acc30'],
            'best_val_acc45': result['best_val_acc45'],
            'total_epochs': result['total_epochs'],
            'training_time_hours': result['training_time_hours'],
            'timestamp': result['timestamp']
        }
        flattened_results.append(flat_result)

    df = pd.DataFrame(flattened_results)
    df = df.sort_values('best_val_mae')

    df.to_excel(excel_file, index=False, engine='openpyxl')
    print(f"\n📊 Results saved to: {excel_file}")

    return excel_file

print("✅ Experiment management functions defined")

# TEST the function
test_params = {
    'learning_rate': 2.0e-04,
    'batch_size': 32,
    'weight_decay': 1e-04,
    'dropout': 0.55
}
test_name = get_experiment_name(test_params)
print(f"✅ Test name: {test_name}")
assert test_name == "lr2e-04_bs32_wd1e-04_dr0.55", f"ERROR: Name is wrong! Got {test_name}"
print(f"✅ Naming function is CORRECT!")

✅ Experiment management functions defined
✅ Test name: lr2e-04_bs32_wd1e-04_dr0.55
✅ Naming function is CORRECT!


## 6.5. 🔍 Validate Grid (Check for Duplicates)

In [7]:
# Validate hyperparameter grid
from collections import Counter

print("🔍 Validating hyperparameter grid...")
print("=" * 70)

param_combinations_test = list(itertools.product(
    HYPERPARAMETER_GRID['learning_rate'],
    HYPERPARAMETER_GRID['batch_size'],
    HYPERPARAMETER_GRID['weight_decay'],
    HYPERPARAMETER_GRID['dropout']
))

print(f"✅ Total combinations: {len(param_combinations_test)}")
print(f"✅ LR values: {sorted(set([x[0] for x in param_combinations_test]))}")

exp_names = []
for lr, bs, wd, dropout in param_combinations_test:
    params = {'learning_rate': lr, 'batch_size': bs, 'weight_decay': wd, 'dropout': dropout}
    exp_name = get_experiment_name(params)
    exp_names.append(exp_name)

name_counts = Counter(exp_names)
duplicates = {name: count for name, count in name_counts.items() if count > 1}

if duplicates:
    print(f"\n❌ ERROR: Found {len(duplicates)} duplicate names!")
    for i, (name, count) in enumerate(list(duplicates.items())[:10], 1):
        print(f"   {i}. {name}: {count} times")
    raise ValueError(f"Fix HYPERPARAMETER_GRID in Cell 2!")
else:
    print(f"\n✅ SUCCESS: All {len(exp_names)} experiment names are UNIQUE!")
    print(f"\n   Sample: {exp_names[0]}, {exp_names[1]}, ...")

print("\n" + "=" * 70)
print("✅ Validation PASSED!")
print("=" * 70)

🔍 Validating hyperparameter grid...
✅ Total combinations: 9
✅ LR values: [0.000207]

✅ SUCCESS: All 9 experiment names are UNIQUE!

   Sample: lr2e-04_bs32_wd1e-04_dr0.40, lr2e-04_bs32_wd1e-04_dr0.42, ...

✅ Validation PASSED!


## 6.6. 🧹 Cleanup Duplicates (Run Once Before Training)

In [8]:
# Clean up duplicate experiments from folder and Excel
import shutil
from collections import defaultdict

print("🧹 CLEANING UP DUPLICATES")
print("=" * 70)

results_dir = Path(BASE_CONFIG['results_dir'])
excel_file = results_dir / 'hyperparameter_tuning_results.xlsx'

# Step 1: Clean Excel
print("\n📊 Step 1: Cleaning Excel...")
if excel_file.exists():
    df = pd.read_excel(excel_file)
    print(f"   Original: {len(df)} experiments")

    df['config'] = df.apply(
        lambda x: f"{x['learning_rate']:.0e}_{x['batch_size']}_{x['weight_decay']:.0e}_{x['dropout']:.2f}",
        axis=1
    )

    duplicates = df[df.duplicated(subset='config', keep='first')]

    if len(duplicates) > 0:
        print(f"   Found {len(duplicates)} duplicates, removing...")
        shutil.copy(excel_file, results_dir / 'results_BACKUP.xlsx')
        df_clean = df.drop_duplicates(subset='config', keep='first').drop(columns=['config'])
        df_clean.to_excel(excel_file, index=False, engine='openpyxl')
        print(f"   ✅ Cleaned: {len(df_clean)} unique experiments")
    else:
        print(f"   ✅ No duplicates in Excel")
else:
    print(f"   ⚠️  Excel not found")

# Step 2: Clean folders
print("\n📁 Step 2: Cleaning folders...")
if results_dir.exists():
    folders = [d for d in results_dir.iterdir() if d.is_dir()]
    print(f"   Found {len(folders)} folders")

    experiments_by_config = defaultdict(list)

    for folder in folders:
        result_file = folder / 'result.json'
        if result_file.exists():
            with open(result_file, 'r') as f:
                result = json.load(f)
                p = result['params']
                key = f"{p['learning_rate']:.0e}_{p['batch_size']}_{p['weight_decay']:.0e}_{p['dropout']:.2f}"
                experiments_by_config[key].append({
                    'folder': folder,
                    'mae': result['best_val_mae']
                })

    to_remove = []
    for key, exps in experiments_by_config.items():
        if len(exps) > 1:
            print(f"   Config {key}: {len(exps)} folders")
            best = min(exps, key=lambda x: x['mae'])
            for exp in exps:
                if exp != best:
                    print(f"      Remove: {exp['folder'].name} (MAE={exp['mae']:.2f}°)")
                    to_remove.append(exp['folder'])
                else:
                    print(f"      Keep:   {exp['folder'].name} (MAE={exp['mae']:.2f}°) ⭐")

    if to_remove:
        print(f"\n   Removing {len(to_remove)} duplicate folders...")
        for folder in to_remove:
            shutil.rmtree(folder)
        print(f"   ✅ Removed {len(to_remove)} folders")
    else:
        print(f"   ✅ No duplicate folders")

print("\n" + "=" * 70)
print("✅ CLEANUP COMPLETE!")
print("=" * 70)

🧹 CLEANING UP DUPLICATES

📊 Step 1: Cleaning Excel...
   ⚠️  Excel not found

📁 Step 2: Cleaning folders...

✅ CLEANUP COMPLETE!


## 7. Main Training Loop

In [9]:
def train_single_experiment(params, train_loader, val_loader, device, results_dir):
    """Train a single hyperparameter configuration"""

    exp_name = get_experiment_name(params)
    exp_dir = Path(results_dir) / exp_name
    exp_dir.mkdir(parents=True, exist_ok=True)

    # Initialize model
    model = PlayerDirectionPredictor(dropout=params['dropout']).to(device)
    criterion = CircularLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=params['learning_rate'],
        weight_decay=params['weight_decay']
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-7
    )

    best_val_mae = float('inf')
    best_val_acc15 = 0
    best_val_acc30 = 0
    best_val_acc45 = 0
    patience_counter = 0
    history = []

    start_time = time.time()

    for epoch in range(FIXED_PARAMS['num_epochs']):
        # Train
        train_loss, train_mae, train_metrics = train_epoch(
            model, train_loader, criterion, optimizer, device,
            use_amp=FIXED_PARAMS['use_amp'],
            accumulation_steps=FIXED_PARAMS['gradient_accumulation_steps']
        )

        # Validate
        val_loss, val_mae, val_metrics = validate(
            model, val_loader, criterion, device,
            use_amp=FIXED_PARAMS['use_amp']
        )

        # Update scheduler
        scheduler.step(val_mae)

        # Save history
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'train_mae': train_mae,
            'train_acc15': train_metrics['acc15'],
            'train_acc30': train_metrics['acc30'],
            'train_acc45': train_metrics['acc45'],
            'val_loss': val_loss,
            'val_mae': val_mae,
            'val_acc15': val_metrics['acc15'],
            'val_acc30': val_metrics['acc30'],
            'val_acc45': val_metrics['acc45'],
            'lr': optimizer.param_groups[0]['lr']
        }
        history.append(epoch_data)

        # Track best metrics
        if val_metrics['acc15'] > best_val_acc15:
            best_val_acc15 = val_metrics['acc15']
        if val_metrics['acc30'] > best_val_acc30:
            best_val_acc30 = val_metrics['acc30']
        if val_metrics['acc45'] > best_val_acc45:
            best_val_acc45 = val_metrics['acc45']

        # Check for improvement
        if val_mae < best_val_mae - FIXED_PARAMS['early_stopping_min_delta']:
            best_val_mae = val_mae
            patience_counter = 0

            # Save best model
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'val_mae': val_mae,
                'val_metrics': val_metrics,
                'params': params
            }, exp_dir / 'best_model.pth')
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= FIXED_PARAMS['early_stopping_patience']:
            break

    training_time = (time.time() - start_time) / 3600

    # Save history
    with open(exp_dir / 'history.json', 'w') as f:
        json.dump(history, f, indent=2)

    # Save final result
    result = {
        'experiment_name': exp_name,
        'params': params,
        'best_val_mae': best_val_mae,
        'best_val_acc15': best_val_acc15,
        'best_val_acc30': best_val_acc30,
        'best_val_acc45': best_val_acc45,
        'total_epochs': len(history),
        'training_time_hours': training_time,
        'completed': True,
        'timestamp': datetime.now().isoformat()
    }

    with open(exp_dir / 'result.json', 'w') as f:
        json.dump(result, f, indent=2)

    return result

print("✅ Main training loop defined")

✅ Main training loop defined


## 8. Test Maximum Batch Size (Optional)

In [10]:
print("🔍 Testing maximum batch size for your GPU...\n")

test_sizes = [32, 64, 96, 128, 192, 256, 320, 384, 448, 512]
max_batch_size = 32

for bs in test_sizes:
    try:
        model = PlayerDirectionPredictor(dropout=0.5).to(device)
        test_batch = torch.randn(bs, 3, 224, 224).to(device)

        with torch.no_grad():
            _ = model(test_batch)

        max_batch_size = bs
        gpu_memory = torch.cuda.max_memory_allocated() / 1e9
        print(f"✅ Batch size {bs:3d}: OK (GPU: {gpu_memory:.1f} GB)")

        del model, test_batch
        torch.cuda.empty_cache()

    except RuntimeError as e:
        if "out of memory" in str(e):
            print(f"❌ Batch size {bs:3d}: Out of memory")
            break
        raise e

print(f"\n🎯 Maximum batch size: {max_batch_size}")
print(f"💡 Your hyperparameter grid includes: {HYPERPARAMETER_GRID['batch_size']}")
print(f"   All batch sizes should work fine!")

🔍 Testing maximum batch size for your GPU...

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 203MB/s]


✅ Batch size  32: OK (GPU: 0.3 GB)
✅ Batch size  64: OK (GPU: 0.6 GB)
✅ Batch size  96: OK (GPU: 0.8 GB)
✅ Batch size 128: OK (GPU: 1.0 GB)
✅ Batch size 192: OK (GPU: 1.5 GB)
✅ Batch size 256: OK (GPU: 2.0 GB)
✅ Batch size 320: OK (GPU: 2.5 GB)
✅ Batch size 384: OK (GPU: 2.9 GB)
✅ Batch size 448: OK (GPU: 3.4 GB)
✅ Batch size 512: OK (GPU: 3.9 GB)

🎯 Maximum batch size: 512
💡 Your hyperparameter grid includes: [32]
   All batch sizes should work fine!


## 9. Load Data (Once)

In [13]:
print("\n" + "="*70)
print("📂 LOADING DATASETS")
print("="*70)

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = PlayerDirectionDataset(BASE_CONFIG['data_root'], 'train', train_transform)
val_dataset = PlayerDirectionDataset(BASE_CONFIG['data_root'], 'val', val_transform)

print(f"\n✅ Train: {len(train_dataset)} images")
print(f"✅ Val: {len(val_dataset)} images")
print("="*70)


📂 LOADING DATASETS

✅ Train: 1400 images
✅ Val: 300 images


## 10. Run Hyperparameter Tuning

In [14]:
# DEBUG: Check for duplicates
print("🔍 Checking for duplicate experiment names...")
from collections import Counter

param_combinations_test = list(itertools.product(
    HYPERPARAMETER_GRID['learning_rate'],
    HYPERPARAMETER_GRID['batch_size'],
    HYPERPARAMETER_GRID['weight_decay'],
    HYPERPARAMETER_GRID['dropout']
))

exp_names = []
for lr, bs, wd, dropout in param_combinations_test:
    params = {'learning_rate': lr, 'batch_size': bs, 'weight_decay': wd, 'dropout': dropout}
    exp_name = get_experiment_name(params)
    exp_names.append(exp_name)

# Check for duplicates
name_counts = Counter(exp_names)
duplicates = {name: count for name, count in name_counts.items() if count > 1}

if duplicates:
    print(f"❌ Found {len(duplicates)} duplicate names:")
    for name, count in list(duplicates.items())[:5]:
        print(f"   {name}: appears {count} times")
else:
    print(f"✅ All {len(exp_names)} experiment names are unique!")

print(f"\n📊 First 5 experiment names:")
for i, name in enumerate(exp_names[:5]):
    print(f"   {i+1}. {name}")

print(f"\n📊 Last 5 experiment names:")
for i, name in enumerate(exp_names[-5:], len(exp_names)-4):
    print(f"   {i}. {name}")
print("\n" + "="*70)
print("🚀 STARTING HYPERPARAMETER TUNING")
print("="*70)

# Create results directory
results_dir = Path(BASE_CONFIG['results_dir'])
results_dir.mkdir(parents=True, exist_ok=True)

# Generate all combinations
param_combinations = list(itertools.product(
    HYPERPARAMETER_GRID['learning_rate'],
    HYPERPARAMETER_GRID['batch_size'],
    HYPERPARAMETER_GRID['weight_decay'],
    HYPERPARAMETER_GRID['dropout']
))

print(f"\n📊 Total combinations to test: {len(param_combinations)}")
print(f"⚙️  Mixed Precision: {FIXED_PARAMS['use_amp']}")
print(f"⚙️  Gradient Accumulation: {FIXED_PARAMS['gradient_accumulation_steps']}x")

# Track all results
all_results = []
completed_count = 0
skipped_count = 0

for idx, (lr, bs, wd, dropout) in enumerate(param_combinations, 1):
    params = {
        'learning_rate': lr,
        'batch_size': bs,
        'weight_decay': wd,
        'dropout': dropout
    }

    exp_name = get_experiment_name(params)

    print(f"\n{'='*70}")
    print(f"🔧 Experiment {idx}/{len(param_combinations)}: {exp_name}")
    print(f"{'='*70}")
    print(f"   LR: {lr:.0e}, Batch: {bs}, WD: {wd:.0e}, Dropout: {dropout}")
    print(f"   Effective Batch: {bs * FIXED_PARAMS['gradient_accumulation_steps']}")

    # Check if already completed
    if check_if_completed(results_dir, exp_name):
        print(f"   ⏭️  Already completed, skipping...")
        skipped_count += 1

        # Load existing result
        with open(results_dir / exp_name / 'result.json', 'r') as f:
            result = json.load(f)
        all_results.append(result)
        continue

    try:
        # Create data loaders with current batch size
        train_loader = DataLoader(
            train_dataset,
            batch_size=bs,
            shuffle=True,
            num_workers=BASE_CONFIG['num_workers'],
            pin_memory=BASE_CONFIG['pin_memory']
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=bs,
            shuffle=False,
            num_workers=BASE_CONFIG['num_workers'],
            pin_memory=BASE_CONFIG['pin_memory']
        )

        # Train
        result = train_single_experiment(
            params, train_loader, val_loader, device, results_dir
        )

        all_results.append(result)
        completed_count += 1

        print(f"\n   ✅ Best Val MAE: {result['best_val_mae']:.2f}°")
        print(f"   ✅ Acc@15°: {result['best_val_acc15']:.1f}%")
        print(f"   ✅ Acc@30°: {result['best_val_acc30']:.1f}%")
        print(f"   ✅ Acc@45°: {result['best_val_acc45']:.1f}%")
        print(f"   ⏱️  Time: {result['training_time_hours']:.2f} hours")

        # Save results to Excel after each experiment
        save_results_to_excel(results_dir, all_results)

        # Clear GPU cache
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"\n   ❌ Error in experiment {exp_name}: {str(e)}")
        continue

print("\n" + "="*70)
print("🎉 HYPERPARAMETER TUNING COMPLETED!")
print("="*70)
print(f"✅ Completed: {completed_count}")
print(f"⏭️  Skipped: {skipped_count}")
print(f"📊 Total results: {len(all_results)}")
print("="*70)

# Final save
if all_results:
    excel_file = save_results_to_excel(results_dir, all_results)

    # Show top 10 results
    sorted_results = sorted(all_results, key=lambda x: x['best_val_mae'])
    print("\n🏆 TOP 10 RESULTS:")
    print("="*70)
    for i, res in enumerate(sorted_results[:10], 1):
        print(f"{i}. {res['experiment_name']}")
        print(f"   MAE: {res['best_val_mae']:.2f}° | Acc@15°: {res['best_val_acc15']:.1f}% | Acc@30°: {res['best_val_acc30']:.1f}%")
    print("="*70)

🔍 Checking for duplicate experiment names...
✅ All 9 experiment names are unique!

📊 First 5 experiment names:
   1. lr2e-04_bs32_wd1e-04_dr0.40
   2. lr2e-04_bs32_wd1e-04_dr0.42
   3. lr2e-04_bs32_wd1e-04_dr0.45
   4. lr2e-04_bs32_wd1e-04_dr0.47
   5. lr2e-04_bs32_wd1e-04_dr0.50

📊 Last 5 experiment names:
   5. lr2e-04_bs32_wd1e-04_dr0.50
   6. lr2e-04_bs32_wd1e-04_dr0.53
   7. lr2e-04_bs32_wd1e-04_dr0.55
   8. lr2e-04_bs32_wd1e-04_dr0.57
   9. lr2e-04_bs32_wd1e-04_dr0.60

🚀 STARTING HYPERPARAMETER TUNING

📊 Total combinations to test: 9
⚙️  Mixed Precision: True
⚙️  Gradient Accumulation: 2x

🔧 Experiment 1/9: lr2e-04_bs32_wd1e-04_dr0.40
   LR: 2e-04, Batch: 32, WD: 1e-04, Dropout: 0.4
   Effective Batch: 64


/tmp/ipython-input-4142847985.py:21: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=use_amp)
/tmp/ipython-input-4142847985.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):
/tmp/ipython-input-4142847985.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=use_amp):



   ✅ Best Val MAE: 60.41°
   ✅ Acc@15°: 36.3%
   ✅ Acc@30°: 56.0%
   ✅ Acc@45°: 73.0%
   ⏱️  Time: 0.27 hours

📊 Results saved to: /content/drive/MyDrive/hyperparameter_tuning_results_round4/hyperparameter_tuning_results.xlsx

🔧 Experiment 2/9: lr2e-04_bs32_wd1e-04_dr0.42
   LR: 2e-04, Batch: 32, WD: 1e-04, Dropout: 0.425
   Effective Batch: 64

   ✅ Best Val MAE: 59.28°
   ✅ Acc@15°: 33.3%
   ✅ Acc@30°: 57.0%
   ✅ Acc@45°: 70.7%
   ⏱️  Time: 0.15 hours

📊 Results saved to: /content/drive/MyDrive/hyperparameter_tuning_results_round4/hyperparameter_tuning_results.xlsx

🔧 Experiment 3/9: lr2e-04_bs32_wd1e-04_dr0.45
   LR: 2e-04, Batch: 32, WD: 1e-04, Dropout: 0.45
   Effective Batch: 64

   ✅ Best Val MAE: 63.95°
   ✅ Acc@15°: 35.0%
   ✅ Acc@30°: 56.0%
   ✅ Acc@45°: 71.0%
   ⏱️  Time: 0.13 hours

📊 Results saved to: /content/drive/MyDrive/hyperparameter_tuning_results_round4/hyperparameter_tuning_results.xlsx

🔧 Experiment 4/9: lr2e-04_bs32_wd1e-04_dr0.47
   LR: 2e-04, Batch: 32, WD: 1e

## 11. Analyze Results

In [ ]:
# Load results from Excel
excel_file = Path(BASE_CONFIG['results_dir']) / 'hyperparameter_tuning_results.xlsx'

if excel_file.exists():
    df = pd.read_excel(excel_file)

    print("\n📊 HYPERPARAMETER ANALYSIS")
    print("="*70)

    # Best overall
    best = df.iloc[0]
    print(f"\n🥇 BEST CONFIGURATION:")
    print(f"   Experiment: {best['experiment_name']}")
    print(f"   LR: {best['learning_rate']:.0e}, Batch: {best['batch_size']}, WD: {best['weight_decay']:.0e}, Dropout: {best['dropout']}")
    print(f"   Val MAE: {best['best_val_mae']:.2f}°")
    print(f"   Acc@15°: {best['best_val_acc15']:.1f}%")
    print(f"   Acc@30°: {best['best_val_acc30']:.1f}%")
    print(f"   Acc@45°: {best['best_val_acc45']:.1f}%")
    print(f"   Training time: {best['training_time_hours']:.2f} hours")

    # Statistics
    print(f"\n📈 OVERALL STATISTICS:")
    print(f"   Mean Val MAE: {df['best_val_mae'].mean():.2f}°")
    print(f"   Std Val MAE: {df['best_val_mae'].std():.2f}°")
    print(f"   Min Val MAE: {df['best_val_mae'].min():.2f}°")
    print(f"   Max Val MAE: {df['best_val_mae'].max():.2f}°")
    print(f"\n   Mean Acc@30°: {df['best_val_acc30'].mean():.1f}%")
    print(f"   Max Acc@30°: {df['best_val_acc30'].max():.1f}%")

    # Best by learning rate
    print(f"\n📊 BEST BY LEARNING RATE:")
    for lr in sorted(df['learning_rate'].unique()):
        lr_df = df[df['learning_rate'] == lr]
        best_lr = lr_df.iloc[0]
        print(f"   LR {lr:.0e}: MAE {best_lr['best_val_mae']:.2f}° | Acc@30° {best_lr['best_val_acc30']:.1f}%")

    # Best by batch size
    print(f"\n📊 BEST BY BATCH SIZE:")
    for bs in sorted(df['batch_size'].unique()):
        bs_df = df[df['batch_size'] == bs]
        best_bs = bs_df.iloc[0]
        print(f"   Batch {bs:3d}: MAE {best_bs['best_val_mae']:.2f}° | Acc@30° {best_bs['best_val_acc30']:.1f}%")

    print("\n" + "="*70)
    print(f"📁 Full results saved to: {excel_file}")
    print("="*70)
else:
    print("⚠️ No results file found. Run the tuning experiment first!")